In [1]:
import pandas as pd
import sqlalchemy
import sqlite3
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
%load_ext sql
%config SqlMagic.autocommit = False
%sql sqlite:///../data/raw/raw_data_project_m1.db

In [3]:
enn = sqlalchemy.create_engine('sqlite:///../data/raw/raw_data_project_m1.db')

personal = pd.read_sql_table(table_name='personal_info', con=enn)

personal.head(4)

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,Male,YES,40_65


In [4]:
personal['age'] =  personal['age'].apply(lambda x: re.sub('\D', '', x)).astype(int)

In [5]:
#from datetime import datetime

def agefunc(x):
    #today = datetime.today().year
    y = 0
    if x <= 1000:
        pass
    else:
        y = 2016 - x
        return y

In [6]:
m = personal['age'] <= 1000
personal['age'] = personal['age'].where(m, personal['age'].apply(agefunc))
personal.head()

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,male,NO,40_65
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,male,yES,40_65
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,male,nO,26_39
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Male,YES,40_65
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Fem,yES,40_65


In [7]:
enn = sqlalchemy.create_engine('sqlite:///../data/raw/raw_data_project_m1.db')

career = pd.read_sql_table(table_name='career_info', con=enn)

career.head(4)

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c


In [8]:
enn = sqlalchemy.create_engine('sqlite:///../data/raw/raw_data_project_m1.db')

country = pd.read_sql_table(table_name='country_info', con=enn)

country.head(4)

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country


In [9]:
enn = sqlalchemy.create_engine('sqlite:///../data/raw/raw_data_project_m1.db')

poll = pd.read_sql_table(table_name='poll_info', con=enn)

poll.head(4)

,uuid,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above


In [10]:
total_data = pd.merge(personal, career, on='uuid')
total_data2 = pd.merge(total_data, country, on='uuid')
glob_data = pd.merge(total_data2, poll, on='uuid')
glob_data.head(5)

,uuid,age,gender,dem_has_children,age_group,dem_education_level,dem_full_time_job,normalized_job_code,country_code,rural,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,male,NO,40_65,no,no,None,AT,countryside,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,male,yES,40_65,high,yes,861a9b9151e11362eb3c77ca914172d0,AT,urban,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,male,nO,26_39,None,no,None,AT,city,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Male,YES,40_65,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,AT,Country,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Fem,yES,40_65,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,AT,city,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...


In [11]:
#import os
#glob_data.to_csv(f'../data/processed/glob_data.csv', index=False)

In [11]:
relevant_data = glob_data[['uuid', 'age', 'country_code', 'normalized_job_code']]
relevant_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9649 entries, 0 to 9648
Data columns (total 4 columns):
uuid                   9649 non-null object
age                    9649 non-null int64
country_code           9649 non-null object
normalized_job_code    5702 non-null object
dtypes: int64(1), object(3)
memory usage: 376.9+ KB


In [12]:
glob_data['country_code'].unique()

array(['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR',
       'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL',
       'PL', 'PT', 'RO', 'SE', 'SI', 'SK'], dtype=object)

In [14]:
relevant_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5702 entries, 1 to 9648
Data columns (total 4 columns):
uuid                   5702 non-null object
age                    5702 non-null int64
country_code           5702 non-null object
normalized_job_code    5702 non-null object
dtypes: int64(1), object(3)
memory usage: 222.7+ KB


In [15]:
relevant_data.head()

,uuid,age,country_code,normalized_job_code
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,AT,861a9b9151e11362eb3c77ca914172d0
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,AT,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,AT,f4b2fb1aa40f661488e2782b6d57ad2f
5,28583060-dbe2-0133-ff9c-0a81e8b09a82,26,AT,27af8700f5577cec835acee2cb90a2ff
7,dd743bb0-d9a5-0133-53b2-0a81e8b09a82,47,AT,c1b670eba9ccb65e7c99f7da116d5b9c


In [38]:
def finding_job(relevant_data, job_title):
    
    #inserted_job = 'Data Scientist'
    
    for code in relevant_data['normalized_job_code']:
        url = f'http://api.dataatwork.org/v1/jobs/' + code
        print(url)
        response = requests.get(url).json()

        if response['title'] == job_title:
            key = response['uuid']
            break
            
    return {key: job_title}
 

In [31]:
data_job = finding_job(relevant_data, 'Data Scientist')

http://api.dataatwork.org/v1/jobs/861a9b9151e11362eb3c77ca914172d0
http://api.dataatwork.org/v1/jobs/049a3f3a2b5f85cb2971ba77ad66e10c
http://api.dataatwork.org/v1/jobs/f4b2fb1aa40f661488e2782b6d57ad2f
http://api.dataatwork.org/v1/jobs/27af8700f5577cec835acee2cb90a2ff
http://api.dataatwork.org/v1/jobs/c1b670eba9ccb65e7c99f7da116d5b9c
http://api.dataatwork.org/v1/jobs/d71c4164c99a5f9bca773e755a72f40f
http://api.dataatwork.org/v1/jobs/2ee80cf30e2483dfe0947adc0428f8c5
http://api.dataatwork.org/v1/jobs/897ef18a9321616495831a4b4b0c1bb2
http://api.dataatwork.org/v1/jobs/a003702c0cea282e17967772dd83de5e
http://api.dataatwork.org/v1/jobs/04c82eae29e1c4a30eac0710b74ab276
http://api.dataatwork.org/v1/jobs/fcaf4aac737e53ad24b9f32f08b8740f
http://api.dataatwork.org/v1/jobs/d71c4164c99a5f9bca773e755a72f40f
http://api.dataatwork.org/v1/jobs/3833bd699e7afe4665b595f282da0387
http://api.dataatwork.org/v1/jobs/19e384e977796c25780d8c28b12089c4
http://api.dataatwork.org/v1/jobs/474b962c13632a04e42a546be4f1

In [52]:
key_job = list(data_job.keys())[0]
data_job


{'8a7a3f1bce0958ae9f090683e5a925b7': 'Data Scientist'}

In [33]:
#def filter_job(x) and replace job code for job name

def filter_job(dataframe, col, value):
    filter_job = dataframe[col] == value
    filtered_data = dataframe[filter_job]
       
    return filtered_data


In [53]:
filtered_data = filter_job(relevant_data, 'normalized_job_code', key_job)
filtered_data['normalized_job_code'].replace(data_job, inplace=True)
filtered_data.head()

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,uuid,age,country_code,normalized_job_code
206,78156750-d949-0133-1a02-0a81e8b09a82,21,BE,Data Scientist
214,92c27100-d9d0-0133-c4db-0a81e8b09a82,43,BE,Data Scientist
249,f30223b0-d950-0133-5b67-0a81e8b09a82,50,BE,Data Scientist
292,5bbb2320-db06-0133-c0ce-0a81e8b09a82,20,BE,Data Scientist
532,5f1b17e0-d97f-0133-f63f-0a81e8b09a82,38,CZ,Data Scientist


In [54]:
##Replace job title
#di_test = {'8a7a3f1bce0958ae9f090683e5a925b7': 'Data Scientist'}
#filtered_data_test = filtered_data
#filtered_data_test['normalized_job_code'].replace(di_test, inplace=True)
#filtered_data_test

In [60]:
def country_name_import(url):

    #Import table
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'lxml')
    table = soup.find_all('td') 
    
    # Scrap countries list
    codes = [i.text.strip() for i in table]
    pre_codes_clean = [i for i in codes if i != ""]
    codes_clean = [re.sub(r'\b[()]|[()]\b', '', code) for code in pre_codes_clean]
    codes_clean[3] = codes_clean[3].replace('EL', 'GR')
    codes_clean[55] = codes_clean[55].replace('UK', 'GB')  
    
    #List to Dict
    countries_codes = {codes_clean[i + 1]: codes_clean[i] for i in range(0, len(codes_clean), 2)} 
    
    return countries_codes

In [61]:
#Replace country codes for country names
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
select_your_country = country_name_import(url)

In [62]:
print(select_your_country) 

{'BE': 'Belgium', 'GR': 'Greece', 'LT': 'Lithuania', 'PT': 'Portugal', 'BG': 'Bulgaria', 'ES': 'Spain', 'LU': 'Luxembourg', 'RO': 'Romania', 'CZ': 'Czechia', 'FR': 'France', 'HU': 'Hungary', 'SI': 'Slovenia', 'DK': 'Denmark', 'HR': 'Croatia', 'MT': 'Malta', 'SK': 'Slovakia', 'DE': 'Germany', 'IT': 'Italy', 'NL': 'Netherlands', 'FI': 'Finland', 'EE': 'Estonia', 'CY': 'Cyprus', 'AT': 'Austria', 'SE': 'Sweden', 'IE': 'Ireland', 'LV': 'Latvia', 'PL': 'Poland', 'GB': 'United Kingdom', 'IS': 'Iceland', 'NO': 'Norway', 'LI': 'Liechtenstein', 'CH': 'Switzerland', 'ME': 'Montenegro', 'MK': 'North Macedonia', 'AL': 'Albania', 'RS': 'Serbia', 'TR': 'Turkey', 'BA': 'Bosnia and Herzegovina', 'XK[2])': 'Kosovo*[1]', 'AM': 'Armenia', 'BY': 'Belarus', 'GE': 'Georgia', 'AZ': 'Azerbaijan', 'MD': 'Moldova', 'UA': 'Ukraine', 'DZ': 'Algeria', 'LB': 'Lebanon', 'SY': 'Syria', 'EG': 'Egypt', 'LY': 'Libya', 'TN': 'Tunisia', 'IL': 'Israel', 'MA': 'Morocco', 'JO': 'Jordan', 'PS': 'Palestine* [3]', 'RU': 'Russia'

In [63]:
filtered_data_country = filter_job(relevant_data, 'country_code', select_your_country['Spain'])
filtered_data['country_code'].replace(select_your_country, inplace=True)
filtered_data_country.head()

KeyError: 'Spain'

In [42]:
def quantity_func(col):
    return col.value_counts()

In [51]:
quantified_data = filtered_data.groupby(['country_code', 'normalized_job_code', 'age'])['normalized_job_code'].agg(quantity_func)
quantified_data.head(20)

country_code  normalized_job_code               age
BE            8a7a3f1bce0958ae9f090683e5a925b7  20     1
                                                21     1
                                                43     1
                                                50     1
CZ            8a7a3f1bce0958ae9f090683e5a925b7  38     1
DE            8a7a3f1bce0958ae9f090683e5a925b7  35     1
                                                36     2
                                                44     1
                                                48     1
                                                49     1
                                                50     1
                                                51     1
DK            8a7a3f1bce0958ae9f090683e5a925b7  25     1
ES            8a7a3f1bce0958ae9f090683e5a925b7  23     1
                                                27     1
                                                35     1
                                    

In [53]:
quantified_data_spain = filtered_data_country.groupby(['country_code', 'normalized_job_code', 'age'])['normalized_job_code'].agg(quantity_func)
quantified_data_spain.head(20)

country_code  normalized_job_code               age
ES            004431b0c1ce29ff6145307fa23f3f98  33     1
                                                38     1
                                                41     1
                                                55     1
              0044971fda4a261b6049a63817913e95  26     1
                                                35     1
                                                42     1
                                                45     1
              022d0e3a40aa3e29bca6851542b27842  64     1
              02e4c028f64f176de2f54bbc2f5403ce  30     1
              049a3f3a2b5f85cb2971ba77ad66e10c  25     1
                                                28     1
                                                39     2
                                                42     1
                                                44     1
                                                45     1
                                    